# Self-attention with trainable weights
**Descrição**  
Nesta etapa, vamos implementar **self-attention com pesos treináveis**, substituindo a versão “fixa” (sem parâmetros) por projeções lineares aprendíveis que geram **queries (Q)**, **keys (K)** e **values (V)**. Isso permite que o modelo aprenda, a partir dos dados, **quais relações entre tokens são mais relevantes** para compor representações contextualizadas. :contentReference[oaicite:0]{index=0}

**Objetivo**  
- Construir uma camada de self-attention **parametrizada** (com matrizes de pesos treináveis).  
- Entender o fluxo completo: **X → (Q, K, V) → scores → pesos de atenção → contexto**.  
- Preparar o terreno para evoluir para **causal attention** e, depois, **multi-head attention**.

**Funcionamento**  
1. **Entrada (X)**: embeddings dos tokens (matriz com formato `[seq_len, d_in]`).  
2. **Projeções treináveis**: aplicamos camadas lineares para obter:
   - `Q = X · W_q`, `K = X · W_k`, `V = X · W_v`  
3. **Similaridade (scores)**: calculamos a afinidade entre tokens via produto escalar:
   - `scores = Q · Kᵀ` (opcionalmente escalado em versões posteriores)  
4. **Normalização (softmax)**: transformamos scores em **pesos de atenção** (distribuição por token).  
5. **Agregação**: combinamos informações ponderadas:
   - `context = attention_weights · V`  
6. **Saída**: representações contextualizadas, onde cada token incorpora informação de outros tokens conforme aprendido pelos pesos.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Para reprodutibilidade
torch.manual_seed(42)

## Frase de exemplo e vocabulário

In [2]:
# Frase de exemplo
sentence = "O gato sobe no tapete".split()

# Vocabulário
vocab = {word: idx for idx, word in enumerate(sentence)}
inv_vocab = {idx: word for word, idx in vocab.items()}

print("Vocabulário:")
for k, v in vocab.items():
    print(f"{k} -> {v}")


Vocabulário:
O -> 0
gato -> 1
sobe -> 2
no -> 3
tapete -> 4


## Conversão para índices (tokens)

In [3]:
# Convertendo palavras para índices
token_ids = torch.tensor([vocab[word] for word in sentence])

print("Tokens:")
for word, idx in zip(sentence, token_ids):
    print(f"{word} -> {idx.item()}")

Tokens:
O -> 0
gato -> 1
sobe -> 2
no -> 3
tapete -> 4


## Camada de Embedding

In [4]:
vocab_size = len(vocab)
embedding_dim = 3  # dimensão pequena para fins didáticos

embedding = nn.Embedding(
    num_embeddings=vocab_size,
    embedding_dim=embedding_dim
)

# Aplicando embedding
X = embedding(token_ids)

print("Shape dos embeddings:", X.shape)
X

Shape dos embeddings: torch.Size([5, 3])


tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863],
        [ 2.2082, -0.6380,  0.4617],
        [ 0.2674,  0.5349,  0.8094],
        [ 1.1103, -1.6898, -0.9890]], grad_fn=<EmbeddingBackward0>)

## Definição da camada de Self-Attention (pesos treináveis)

In [5]:
class SelfAttention(nn.Module):
    def __init__(self, d_in: int, d_attn: int):
        super().__init__()
        
        self.W_q = nn.Linear(d_in, d_attn, bias=False)
        self.W_k = nn.Linear(d_in, d_attn, bias=False)
        self.W_v = nn.Linear(d_in, d_attn, bias=False)

    def forward(self, x: torch.Tensor):
        """
        x: Tensor de shape [seq_len, d_in]
        """
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)

        # Produto escalar entre queries e keys
        scores = Q @ K.T

        # Softmax para obter pesos de atenção
        attn_weights = F.softmax(scores, dim=-1)

        # Combinação ponderada dos values
        context = attn_weights @ V

        return context, attn_weights


In [6]:
d_attn = 3

self_attention = SelfAttention(
    d_in=embedding_dim,
    d_attn=d_attn
)

context, attn_weights = self_attention(X)

print("Shape do contexto:", context.shape)
print("Shape dos pesos de atenção:", attn_weights.shape)

Shape do contexto: torch.Size([5, 3])
Shape dos pesos de atenção: torch.Size([5, 5])


## Visualizando os pesos de atenção (parte mais didática)

Aqui mostramos quem presta atenção em quem.

In [7]:
print("Pesos de atenção:\n")

for i, word in enumerate(sentence):
    print(f"Palavra: '{word}'")
    for j, weight in enumerate(attn_weights[i]):
        print(f"  → atenção em '{sentence[j]}': {weight.item():.4f}")
    print()


Pesos de atenção:

Palavra: 'O'
  → atenção em 'O': 0.1942
  → atenção em 'gato': 0.1878
  → atenção em 'sobe': 0.2300
  → atenção em 'no': 0.2064
  → atenção em 'tapete': 0.1816

Palavra: 'gato'
  → atenção em 'O': 0.2115
  → atenção em 'gato': 0.1965
  → atenção em 'sobe': 0.1864
  → atenção em 'no': 0.2089
  → atenção em 'tapete': 0.1967

Palavra: 'sobe'
  → atenção em 'O': 0.1889
  → atenção em 'gato': 0.1312
  → atenção em 'sobe': 0.3133
  → atenção em 'no': 0.2741
  → atenção em 'tapete': 0.0926

Palavra: 'no'
  → atenção em 'O': 0.1720
  → atenção em 'gato': 0.1814
  → atenção em 'sobe': 0.2707
  → atenção em 'no': 0.1786
  → atenção em 'tapete': 0.1973

Palavra: 'tapete'
  → atenção em 'O': 0.2458
  → atenção em 'gato': 0.1744
  → atenção em 'sobe': 0.1636
  → atenção em 'no': 0.2930
  → atenção em 'tapete': 0.1232



### O que aprendemos aqui?

- Cada palavra gera sua própria **Query, Key e Value**
- A atenção é calculada comparando **Query × Key**
- O softmax transforma similaridade em **distribuição de atenção**
- O contexto final é uma **soma ponderada dos Values**
- Os pesos `W_q`, `W_k`, `W_v` são **treináveis**, permitindo ao modelo aprender relações semânticas
